<a href="https://colab.research.google.com/github/Nainanjali7/Book-Recommendation-system/blob/main/Book_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [5]:
books = pd.read_csv('books.csv')
users = pd.read_csv('users.csv')
ratings = pd.read_csv('ratings.csv')

In [6]:
books['Image-URL-M'][1]

'http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg'

In [7]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [8]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [9]:
print(books.shape)
print(ratings.shape)
print(users.shape)

(58972, 8)
(314591, 3)
(278858, 3)


In [10]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
Image-URL-S            0
Image-URL-M            0
Image-URL-L            1
dtype: int64

In [11]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [12]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    1
dtype: int64

In [13]:
books.duplicated().sum()

0

In [14]:
ratings.duplicated().sum()

14

In [15]:
users.duplicated().sum()

0

##Popularity Based Recommender System

In [16]:
ratings_with_name = ratings.merge(books,on='ISBN')

In [17]:
num_rating_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)
num_rating_df

,Book-Title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,3
1,Beyond IBM: Leadership Marketing and Finance ...,1
2,Earth Prayers From around the World: 365 Pray...,2
3,Final Fantasy Anthology: Official Strategy Gu...,3
4,It Takes Two,2
...,...,...
52363,Â¿QuiÃ©n se ha llevado mi queso?,1
52364,"Â¿QuÃ© me quieres, amor?",2
52365,Ã?Â?ber die Freiheit.,1
52366,Ã?Â?bermorgen.,1


In [18]:
avg_rating_df = ratings_with_name.groupby('Book-Title').mean()['Book-Rating'].reset_index()
avg_rating_df.rename(columns={'Book-Rating':'avg_rating'},inplace=True)
avg_rating_df

<ipython-input-18-ebeed7a91fd4>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg_rating_df = ratings_with_name.groupby('Book-Title').mean()['Book-Rating'].reset_index()


,Book-Title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,0.000000
1,Beyond IBM: Leadership Marketing and Finance ...,0.000000
2,Earth Prayers From around the World: 365 Pray...,9.500000
3,Final Fantasy Anthology: Official Strategy Gu...,3.333333
4,It Takes Two,0.000000
...,...,...
52363,Â¿QuiÃ©n se ha llevado mi queso?,7.000000
52364,"Â¿QuÃ© me quieres, amor?",3.000000
52365,Ã?Â?ber die Freiheit.,7.000000
52366,Ã?Â?bermorgen.,0.000000


In [19]:
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df

,Book-Title,num_ratings,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,3,0.000000
1,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
2,Earth Prayers From around the World: 365 Pray...,2,9.500000
3,Final Fantasy Anthology: Official Strategy Gu...,3,3.333333
4,It Takes Two,2,0.000000
...,...,...,...
52363,Â¿QuiÃ©n se ha llevado mi queso?,1,7.000000
52364,"Â¿QuÃ© me quieres, amor?",2,3.000000
52365,Ã?Â?ber die Freiheit.,1,7.000000
52366,Ã?Â?bermorgen.,1,0.000000


In [20]:
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_rating',ascending=False).head(50)

In [21]:
popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_rating']]

In [22]:
popular_df['Image-URL-M'][0]

'http://images.amazon.com/images/P/0385504209.01.MZZZZZZZ.jpg'

##Collaborative Based Recommender System

In [24]:
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200
educated_users = x[x].index

In [25]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(educated_users)]

In [26]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index

In [27]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [28]:
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [29]:
pt.fillna(0,inplace=True)

In [30]:
pt

User-ID,254,2766,2977,3363,4017,6251,6543,6563,6575,7158,...,69078,69232,69355,69697,69971,70052,73394,75591,277427,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
Bridget Jones's Diary,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,2.666667,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0
Divine Secrets of the Ya-Ya Sisterhood: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Lovely Bones: A Novel,0.0,9.0,0.0,0.0,10.0,0.0,0.0,0.0,0.000000,0.0,...,7.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
The Notebook,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Pelican Brief,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
The Secret Life of Bees,9.0,0.0,0.0,0.0,0.0,8.0,9.0,9.0,8.000000,0.0,...,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0
Wild Animus,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
similarity_scores = cosine_similarity(pt)

In [33]:
similarity_scores.shape

(7, 7)

In [47]:
def recommend(book_name):
    # index fetch
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:5]

    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))

        data.append(item)

    return data

In [49]:
recommend('The Secret Life of Bees')

[['Divine Secrets of the Ya-Ya Sisterhood: A Novel',
  'Rebecca Wells',
  'http://images.amazon.com/images/P/0060928336.01.MZZZZZZZ.jpg'],
 ['The Notebook',
  'Nicholas Sparks',
  'http://images.amazon.com/images/P/0446605239.01.MZZZZZZZ.jpg'],
 ['The Lovely Bones: A Novel',
  'Alice Sebold',
  'http://images.amazon.com/images/P/0316666343.01.MZZZZZZZ.jpg'],
 ["Bridget Jones's Diary",
  'Helen Fielding',
  'http://images.amazon.com/images/P/0330332775.01.MZZZZZZZ.jpg']]

In [52]:
pt.index[3]

'The Notebook'

In [53]:
import pickle
pickle.dump(popular_df,open('popular.pkl','wb'))

In [54]:
books.drop_duplicates('Book-Title')

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
58967,0517569035,Catlore,Desmond Morris,1988,Random House Inc,http://images.amazon.com/images/P/0517569035.0...,http://images.amazon.com/images/P/0517569035.0...,http://images.amazon.com/images/P/0517569035.0...
58968,0451202724,Murder of a Sweet Old Lady,Denise Swanson,2001,Signet Book,http://images.amazon.com/images/P/0451202724.0...,http://images.amazon.com/images/P/0451202724.0...,http://images.amazon.com/images/P/0451202724.0...
58969,0812590155,Help Me Please,Barbara D'Amato,2001,Forge,http://images.amazon.com/images/P/0812590155.0...,http://images.amazon.com/images/P/0812590155.0...,http://images.amazon.com/images/P/0812590155.0...
58970,0451405617,Closely Akin to Murder: A Claire Malloy Myster...,Joan Hess,1997,Onyx Books,http://images.amazon.com/images/P/0451405617.0...,http://images.amazon.com/images/P/0451405617.0...,http://images.amazon.com/images/P/0451405617.0...


In [55]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(similarity_scores,open('similarity_scores.pkl','wb'))